In [44]:
# Load pandas library
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [45]:
%matplotlib inline


In [ ]:
df1_import = pd.read_csv('atlantic_sunrise2021.csv',  
                           sep=',',
                           engine='python',
                           na_values = ['NA', 'N/A','NaN'],
                           header=None,
                           skiprows = [0],
                           names = ['customer_id', 'First_Name', 'last_name', 'address', 'City', 'county', 'state',
                           'ZIP', 'customer_phone', 'Fax_Number', 'Customer_Email', 'Rental_ID', 'Weekly_Rate', 'Parking',
                           'Pool_Access', 'Repeat_Customer', 'Bedrooms', 'MGMT_Discount', 'Date'])  # Read first CSV file)
                           
df1_import.head().append(df1_import.tail())

In [ ]:
df1_import.duplicated()

In [ ]:
df1_import.duplicated(keep="last")

In [ ]:
len(df1_import), len(df1_import.drop_duplicates())

In [ ]:
df1_import = df1_import.drop_duplicates()
df1_import

In [ ]:
df1_import = pd.DataFrame(df1_import)
df1_import.dtypes

In [ ]:
df1_import.columns = map(str.lower,df1_import.columns)

df1_import.columns = map(lambda x: x.replace("-", "_").replace(" ", "_"), df1_import.columns)

df1_import.columns

In [53]:
convert_data = pd.DataFrame(df1_import)
cols = ['customer_id', 'first_name', 'last_name', 'address', 'city', 'county','state','zip',
        'customer_phone', 'fax_number', 'customer_email', 'rental_id','bedrooms']
convert_data[cols] = convert_data[cols].astype('str')

In [54]:
for col in ['parking','pool_access','repeat_customer', 'mgmt_discount']:
    df1_import[col] = df1_import[col].astype('bool_')

In [ ]:
df1_import[['date']].apply(lambda x: x.astype('datetime64[ns]'))

In [ ]:
df1_import["weekly_rate"] = pd.to_numeric(df1_import.weekly_rate, errors = 'coerce', downcast ='integer' )
df1_import.dtypes

In [ ]:
df1_import.astype

In [ ]:
df2_import = pd.read_csv('atlantic_sunrise2022.csv',
                           sep=',',
                           engine='python',
                           na_values = ['NA', 'N/A', 'NaN'],
                           header=None,
                           skiprows = [0],
                           names = ['customer_id', 'First_Name', 'last_name', 'address', 'City', 'county', 'state',
                           'ZIP', 'customer_phone', 'Fax_Number', 'currency', 'Customer_Email', 'Rental_ID', 'Weekly_Rate', 'Parking',
                           'Pool_Access', 'Repeat_Customer','Bedrooms', 'MGMT_Discount', 'Date'])    # Read second CSV file

df2_import.head().append(df2_import.tail())

In [ ]:
df2_import.duplicated()

In [ ]:
df2_import.duplicated(keep="last")

In [ ]:
len(df2_import), len(df2_import.drop_duplicates())

In [ ]:
df2_import = df2_import.drop_duplicates()
df2_import

In [ ]:
df2_import = pd.DataFrame(df2_import)
df2_import.dtypes

In [ ]:
df2_import.columns = map(str.lower,df2_import.columns)

df2_import.columns = map(lambda x: x.replace("-", "_").replace(" ", "_"), df2_import.columns)

df2_import.columns

In [65]:
convert_data = pd.DataFrame(df2_import)
cols = ['customer_id', 'first_name', 'last_name', 'address', 'city', 'county','state','zip',
        'customer_phone', 'fax_number', 'currency', 'customer_email', 'rental_id','bedrooms']
convert_data[cols] = convert_data[cols].astype('str')

In [66]:
for col in ['parking','pool_access','repeat_customer', 'mgmt_discount']:
    df2_import[col] = df2_import[col].astype('bool_')

In [ ]:
df2_import[['date']].apply(lambda x: x.astype('datetime64[ns]'))

In [68]:
df2_import= df2_import.astype({"weekly_rate": int},errors='ignore')

In [ ]:
df2_import.astype

In [ ]:
resort_df = df1_import.merge(df2_import,
                              how='outer', 
                              right_index= False,
                              left_index= False)

resort_df

In [71]:
resort_df.fillna('USD', inplace=True)

In [ ]:
resort_df["currency"] = resort_df['currency'].replace('None',  value='USD')
resort_df


In [ ]:
resort_df.rental_id.unique()

In [ ]:
for col in resort_df.columns:
    if 'rental_id' in col:
        del resort_df[col]

resort_df

In [ ]:
resort_df.insert(loc=12, column='rent_discount', value='0.15', allow_duplicates=True)

resort_df

In [76]:
type(True)

bool

In [77]:
type(False)

bool

In [78]:
issubclass(bool, int)

True

In [79]:
True == 1

True

In [80]:
False == 0

True